In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
!pip install transformers datasets peft accelerate
!pip install bitsandbytes accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

# 기본 A.X.4.0-Light

In [8]:
!pip install -q transformers accelerate google-generativeai

In [ ]:
import os, re, time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# GPU 메모리/속도 유틸
def fmt_mb(x): return f"{x / (1024**2):.2f} MB"

def print_gpu_usage(tag):
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        alloc = torch.cuda.memory_allocated()
        reserve = torch.cuda.memory_reserved()
        peak = torch.cuda.max_memory_allocated()
        print(f"- [GPU] {tag} 사용량: {fmt_mb(alloc)}")
        print(f"- [GPU] {tag} 예약량: {fmt_mb(reserve)}")
        print(f"- [GPU] {tag} 최대 사용량: {fmt_mb(peak)}")
    else:
        print(f"- [GPU] {tag}: GPU 미사용")

# A.X-4.0-Light - 기본
class AX4LightGenerator:
    def __init__(self, model_name: str = "skt/A.X-4.0-Light"):
        if torch.cuda.is_available():
            print("사용 중인 GPU:", torch.cuda.get_device_name(0))
            torch.cuda.reset_peak_memory_stats()

        t0 = time.time()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, trust_remote_code=True)
        if self.tokenizer.pad_token_id is None and self.tokenizer.eos_token_id is not None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        # fp16 + device_map="auto"
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            trust_remote_code=True,
        )
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        print(f"[속도] 모델 로드 시간: {time.time()-t0:.2f}초")
        print_gpu_usage("모델 로드 후")

    def make_prompt(self, name: str, age: int) -> str:
        return f"""### 질문:
{age}살 아이 '{name}'을 위한 짧은 동화책을 만들어줘.
문장은 간단하고, 유아가 이해할 수 있는 수준이어야 해.
내용은 따뜻하고 교훈적인 이야기면 좋아.
처음에는 "옛날 옛적에"로 시작해줘.

### 답변:
"""

    @torch.inference_mode()
    def generate(self, name: str, age: int, max_new_tokens: int = 300, temperature: float = 0.8, top_p: float = 0.9) -> dict:
        prompt = self.make_prompt(name, age)
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        if torch.cuda.is_available():
            torch.cuda.reset_peak_memory_stats()

        t0 = time.time()
        output_ids = self.model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=1.05,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.pad_token_id,
        )
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        dt = time.time() - t0

        # 생성 토큰수/속도
        input_len = inputs["input_ids"].shape[1]
        total_len = output_ids[0].shape[0]
        new_tokens = max(0, total_len - input_len)
        tps = (new_tokens / dt) if dt > 0 else float("nan")

        print(f"\n동화 생성 시간: {dt:.2f}초 | 생성 토큰: {new_tokens} | 속도: {tps:.2f} tok/s")
        print_gpu_usage("생성 후")

        # 디코드 & 프롬프트 제거
        text = self.tokenizer.decode(output_ids[0], skip_special_tokens=True)
        if "### 답변:" in text:
            story = text.split("### 답변:", 1)[-1].strip()
        else:
            # 백업: 프롬프트 길이로 자르기
            story = text[len(prompt):].lstrip() if text.startswith(prompt) else text
        return {"story": story, "gen_time_s": dt, "new_tokens": new_tokens, "tps": tps}

# Gemini 평가
def evaluate_with_gemini(story: str) -> dict:
    """
    7개 항목 각각 10점 만점 점수를 담은 텍스트를 받으면,
    점수들을 파싱하여 평균(=총점, 10점 만점)을 계산합니다.
    환경변수 GEMINI_API_KEY 사용.
    """
    import google.generativeai as genai

    genai.configure(api_key="your_api_key")



    model = genai.GenerativeModel("gemini-2.5-flash")
    prompt = f"""
다음은 유아를 위한 동화책 내용입니다. 아래 기준에 따라 10점 만점으로 평가해주세요.

평가 기준:
1. 키워드 반영 여부
2. 동화 맥락의 일관성
3. 교훈의 명확성
4. 문법적 완성도
5. 비속어나 부적절한 표현의 유무
6. 서사 전개의 논리성
7. 반복되는 문장의 빈도

형식:
1. 키워드 반영 여부: X점 – [간단한 이유]
2. 동화 맥락의 일관성: X점 – ...
3. 교훈의 명확성: X점 – ...
4. 문법적 완성도: X점 – ...
5. 비속어나 부적절한 표현의 유무: X점 – ...
6. 서사 전개의 논리성: X점 – ...
7. 반복되는 문장의 빈도: X점 – ...

[동화 내용]
{story}
"""

    t0 = time.time()
    resp = model.generate_content(prompt)
    eval_time = time.time() - t0

    text = resp.text or ""
    # 숫자 파싱
    scores = [float(s) for s in re.findall(r"(\d+(?:\.\d+)?)\s*점", text)]
    # 7개 항목만 사용
    scores = scores[:7] if len(scores) >= 7 else scores + [0.0] * (7 - len(scores))

    avg_10 = sum(scores) / 7.0  # 10점 만점 평균
    return {
        "raw_text": text,
        "scores": scores,
        "avg_10": avg_10,
        "eval_time_s": eval_time
    }

# 실행
if __name__ == "__main__":
    name, age = "지훈", 6

    gen = AX4LightGenerator("skt/A.X-4.0-Light")
    result = gen.generate(name=name, age=age, max_new_tokens=320)
    story = result["story"]

    print("\n==== 동화 내용 ====\n")
    print(story[:1500] + ("\n...\n" if len(story) > 1500 else ""))

    # Gemini 평가
    try:
        eval_res = evaluate_with_gemini(story)
        # 요약 출력
        print("\n==== Gemini 평가 요약 ====")
        for i, sc in enumerate(eval_res["scores"], 1):
            print(f"{i}. 항목 점수: {sc} / 10")
        print(f"- 평균 총점(10점 만점): {eval_res['avg_10']:.2f}")
        print(f"- 평가 시간: {eval_res['eval_time_s']:.2f}초")
    except Exception as e:
        print("[평가 오류]", e)

    # 성능 리포트
    print("\n==== 성능 리포트 ====")
    print(f"- 생성 시간: {result['gen_time_s']:.2f}초")
    print(f"- 생성 토큰 수: {result['new_tokens']}")
    print(f"- 생성 속도: {result['tps']:.2f} tok/s")

사용 중인 GPU: NVIDIA A100-SXM4-40GB


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[속도] 모델 로드 시간: 4.85초
- [GPU] 모델 로드 후 사용량: 33768.45 MB
- [GPU] 모델 로드 후 예약량: 33978.00 MB
- [GPU] 모델 로드 후 최대 사용량: 33841.92 MB

동화 생성 시간: 7.91초 | 생성 토큰: 247 | 속도: 31.23 tok/s
- [GPU] 생성 후 사용량: 19918.81 MB
- [GPU] 생성 후 예약량: 33980.00 MB
- [GPU] 생성 후 최대 사용량: 19940.96 MB

==== 동화 내용 ====

옛날 옛적에 작은 마을에 지훈이라는 아이가 있었어요. 지훈은 항상 새로운 것을 배우고 싶었어요. 어느 날, 마을 근처에 커다란 나무가 있었어요. 그 나무는 아주 오래되고, 많은 동물들이 그 나무 아래서 쉬곤 했어요.

지훈은 그 나무에 대해 궁금해했어요. 그래서 나무에게 물었어요. "나무씨, 당신은 얼마나 오래 살았어요?" 나무는 속삭이듯 대답했어요. "나는 100년 이상 살았어. 많은 이야기를 알고 있어."

지훈은 나무에게 더 많은 이야기를 듣고 싶었어요. 그래서 매일 나무 주변에 앉아 이야기를 들었어요. 시간이 지나면서 지훈은 나무에게서 자연의 소중함과 친구들과의 우정을 배웠어요.

어느 날, 마을에 큰 폭풍이 몰아쳤어요. 지훈은 나무 아래에 가서 보호받았어요. 나무는 지훈에게 말했어요. "너는 항상 나를 찾아와 주었어. 이제 너도 다른 친구들을 도와줘야 해."

지훈은 나무 덕분에 많은 것을 배우고, 다른 친구들을 도와주기로 결심했어요. 그는 친구들과 함께 나무를 심고, 자연을 보호하는 방법을 배웠어요.

그리고 지훈은 이렇게 생각했어요. "작은 행동들이 모여 큰 변화를 만들 수 있어."

그래서 지훈은 매일 조금씩 노력하며, 따뜻한 마음을 가진 아이로 자랐어요. 그리고 마을 사람들은 지훈을 자랑스러워했답니다.

끝.

==== Gemini 평가 요약 ====
1. 항목 점수: 9.0 / 10
2. 항목 점수: 10.0 / 10
3. 항목 점수: 10.0 /

# QLoRA

In [7]:
import os
import json
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

# 환경 설정
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()
torch.set_float32_matmul_precision("high")  # matmul 최적화

# 설정
model_name = "skt/A.X-4.0-Light"
data_path = "/content/gdrive/MyDrive/Colab Notebooks/ING/모델 평가/dataset/a4_sft.jsonl"
output_dir = "/content/gdrive/MyDrive/Colab Notebooks/ING/모델 평가/model/Qlora-ax4"
max_length = 512
SEED = 42

# 토크나이저
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# pad_token 없으면 EOS로 대체
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 데이터 유틸
def clean_roles(messages):
    valid_roles = {"system", "user", "assistant"}
    return [m for m in messages if m["role"] in valid_roles and isinstance(m.get("content"), str)]

def load_chat_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        return [{"messages": clean_roles(json.loads(line)["messages"])} for line in f]

def split_if_too_long(example):
    messages = example["messages"]
    sys_msgs = [m for m in messages if m["role"] == "system"]
    user_msgs = [m for m in messages if m["role"] == "user"]
    assistant_msgs = [m for m in messages if m["role"] == "assistant"]

    if not assistant_msgs:
        return []

    content = assistant_msgs[0]["content"]
    paragraphs = [p.strip() for p in content.split("\n") if p.strip()]

    chunks = []
    current_chunk = []

    for p in paragraphs:
        current_chunk.append(p)
        new_assistant = {"role": "assistant", "content": "\n".join(current_chunk)}
        temp = sys_msgs + user_msgs + [new_assistant]

        try:
            prompt = tokenizer.apply_chat_template(temp, add_generation_prompt=False, return_tensors=None)
            if isinstance(prompt, list):
                prompt = "".join(map(str, prompt))
            token_ids = tokenizer(prompt)["input_ids"]
        except Exception:
            continue

        if len(token_ids) > max_length:
            current_chunk.pop()
            if current_chunk:
                new_assistant = {"role": "assistant", "content": "\n".join(current_chunk)}
                chunks.append({"messages": sys_msgs + user_msgs + [new_assistant]})
            current_chunk = [p]

    if current_chunk:
        new_assistant = {"role": "assistant", "content": "\n".join(current_chunk)}
        chunks.append({"messages": sys_msgs + user_msgs + [new_assistant]})

    return chunks

# 토크나이즈
def tokenize_chat(example):
    # 전체 시퀀스
    full_text = tokenizer.apply_chat_template(
        example["messages"], add_generation_prompt=False, return_tensors=None
    ) or ""
    if isinstance(full_text, list):
        full_text = "".join(map(str, full_text))

    # 2) 프리픽스
    prefix_messages = [m for m in example["messages"] if m["role"] != "assistant"]
    prefix_text = tokenizer.apply_chat_template(
        prefix_messages, add_generation_prompt=True, return_tensors=None
    ) or ""
    if isinstance(prefix_text, list):
        prefix_text = "".join(map(str, prefix_text))

    # 토크나이즈
    enc = tokenizer(full_text, padding="max_length", truncation=True, max_length=max_length)
    input_ids = enc["input_ids"]
    attn = enc["attention_mask"]

    # prefix 길이 산출
    prefix_ids = tokenizer(prefix_text, truncation=True, max_length=max_length)["input_ids"]
    prefix_len = len(prefix_ids)

    # 라벨 생성
    labels = input_ids.copy()

    # 프리픽스 -100
    for i in range(min(prefix_len, max_length)):
        labels[i] = -100

    # 패딩 -100
    pad_id = tokenizer.pad_token_id
    for i, tok in enumerate(input_ids):
        if attn[i] == 0 or tok == pad_id:
            labels[i] = -100

    return {
        "input_ids": input_ids,
        "attention_mask": attn,
        "labels": labels
    }

# QLoRA 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# QLoRA 준비
base_model = prepare_model_for_kbit_training(base_model)

# 그래디언트 체크포인팅 + 캐시 비활성화
base_model.gradient_checkpointing_enable()
if hasattr(base_model, "config"):
    base_model.config.use_cache = False

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "v_proj"]
)

model = get_peft_model(base_model, peft_config)

# 데이터 처리
raw_data = load_chat_jsonl(data_path)
split_data = []
for ex in raw_data:
    split_data.extend(split_if_too_long(ex))

dataset = Dataset.from_list(split_data)
print(f"분할 후 샘플 수: {len(dataset)}")

tokenized_dataset = dataset.map(
    tokenize_chat,
    batched=False,
    remove_columns=["messages"]
)

if len(tokenized_dataset) == 0:
    raise ValueError("학습할 샘플이 없습니다. max_length를 늘리거나 데이터를 확인하세요.")
print(f"최종 학습 샘플 수: {len(tokenized_dataset)}")

# 학습 설정
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    num_train_epochs=3,
    bf16=True,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    remove_unused_columns=False,
    report_to="none",

    # 안정화
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,             # 전체 스텝의 5% 워밍업
    max_grad_norm=1.0,             # 그래디언트 클리핑
    seed=SEED,
    data_seed=SEED,
    group_by_length=True,          # 길이 비슷한 샘플끼리 배치 → 패딩 낭비 감소
    dataloader_num_workers=2,      # I/O 병목 완화(환경 가능 시)
)

from transformers import default_data_collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator  # labels 이미 준비했으므로 기본 콜레이터 사용
)

# 실행 & 저장
torch.cuda.empty_cache()
trainer.train()
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (17878 > 16384). Running this sequence through the model will result in indexing errors


분할 후 샘플 수: 1752


Map:   0%|          | 0/1752 [00:00<?, ? examples/s]

최종 학습 샘플 수: 1752


/tmp/ipython-input-2229153917.py:204: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,4.923800
20,4.897900
30,4.954700
40,4.982900
50,4.938500
60,4.831700
70,4.886100
80,4.766900
90,4.773600
100,4.728600


Step,Training Loss
10,4.923800
20,4.897900
30,4.954700
40,4.982900
50,4.938500
60,4.831700
70,4.886100
80,4.766900
90,4.773600
100,4.728600


('/content/gdrive/MyDrive/Colab Notebooks/ING/모델 평가/model/Qlora-ax4/tokenizer_config.json',
 '/content/gdrive/MyDrive/Colab Notebooks/ING/모델 평가/model/Qlora-ax4/special_tokens_map.json',
 '/content/gdrive/MyDrive/Colab Notebooks/ING/모델 평가/model/Qlora-ax4/chat_template.jinja',
 '/content/gdrive/MyDrive/Colab Notebooks/ING/모델 평가/model/Qlora-ax4/vocab.json',
 '/content/gdrive/MyDrive/Colab Notebooks/ING/모델 평가/model/Qlora-ax4/merges.txt',
 '/content/gdrive/MyDrive/Colab Notebooks/ING/모델 평가/model/Qlora-ax4/added_tokens.json',
 '/content/gdrive/MyDrive/Colab Notebooks/ING/모델 평가/model/Qlora-ax4/tokenizer.json')

In [ ]:
import os, re, time, torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel


BASE_MODEL = "skt/A.X-4.0-Light"
LORA_PATH  = "/content/gdrive/MyDrive/Colab Notebooks/ING/모델 평가/model/Qlora-ax4/checkpoint-2628"

assert os.path.isdir(LORA_PATH), f"QLoRA 경로가 존재하지 않습니다: {LORA_PATH}"

# GPU 메모리/속도 유틸
def fmt_mb(x): return f"{x / (1024**2):.2f} MB"

def print_gpu_usage(tag):
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        alloc = torch.cuda.memory_allocated()
        reserve = torch.cuda.memory_reserved()
        peak = torch.cuda.max_memory_allocated()
        print(f"- [GPU] {tag} 사용량: {fmt_mb(alloc)}")
        print(f"- [GPU] {tag} 예약량: {fmt_mb(reserve)}")
        print(f"- [GPU] 최대 사용량: {fmt_mb(peak)}")
    else:
        print(f"- [GPU] {tag}: GPU 미사용")


# A.X-4.0-Light + QLoRA 어댑터(4bit)
class AX4QLoRA:
    def __init__(self, base_model: str = BASE_MODEL, lora_path: str = LORA_PATH):
        if torch.cuda.is_available():
            print("사용 중인 GPU:", torch.cuda.get_device_name(0))
            torch.cuda.reset_peak_memory_stats()

        t0 = time.time()
        # 토크나이저
        self.tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True, trust_remote_code=True)
        if self.tokenizer.pad_token_id is None and self.tokenizer.eos_token_id is not None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        # 4bit 양자화(QLoRA 추론용)
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
        )

        # 베이스 모델(4bit) 로드
        base = AutoModelForCausalLM.from_pretrained(
            base_model,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True,
        )

        # LoRA 어댑터 장착
        self.model = PeftModel.from_pretrained(base, lora_path)
        self.model.eval()

        if torch.cuda.is_available():
            torch.cuda.synchronize()
        print(f"[속도] 모델 로드 시간: {time.time()-t0:.2f}초")
        print_gpu_usage("모델 로드 후")

    def make_prompt(self, name: str, age: int) -> str:
        return f"""### 질문:
{age}살 아이 '{name}'을 위한 짧은 동화책을 만들어줘.
문장은 간단하고, 유아가 이해할 수 있는 수준이어야 해.
내용은 따뜻하고 교훈적인 이야기면 좋아.
처음에는 "옛날 옛적에"로 시작해줘.

### 답변:
"""

    @torch.inference_mode()
    def generate(self, name: str, age: int, max_new_tokens: int = 320, temperature: float = 0.8, top_p: float = 0.9):
        prompt = self.make_prompt(name, age)
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        if torch.cuda.is_available():
            torch.cuda.reset_peak_memory_stats()

        t0 = time.time()
        output_ids = self.model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=1.05,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.pad_token_id,
        )
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        dt = time.time() - t0

        # 생성 토큰수/속도
        input_len = inputs["input_ids"].shape[1]
        total_len = output_ids[0].shape[0]
        new_tokens = max(0, total_len - input_len)
        tps = (new_tokens / dt) if dt > 0 else float("nan")

        print(f"\n동화 생성 시간: {dt:.2f}초 | 생성 토큰: {new_tokens} | 속도: {tps:.2f} tok/s")
        print_gpu_usage("생성 후")

        # 디코드 & 프롬프트 제거
        text = self.tokenizer.decode(output_ids[0], skip_special_tokens=True)
        if "### 답변:" in text:
            story = text.split("### 답변:", 1)[-1].strip()
        else:
            story = text[len(prompt):].lstrip() if text.startswith(prompt) else text

        return {"story": story, "gen_time_s": dt, "new_tokens": new_tokens, "tps": tps}

# Gemini 평가
def evaluate_with_gemini(story: str) -> dict:
    """
    7개 항목 각각 10점 만점 점수를 담은 텍스트를 받으면,
    점수들을 파싱하여 평균(=총점, 10점 만점)을 계산합니다.
    환경변수 GEMINI_API_KEY 사용.
    """
    import google.generativeai as genai

    genai.configure(api_key=" ")



    model = genai.GenerativeModel("gemini-2.5-flash")
    prompt = f"""
다음은 유아를 위한 동화책 내용입니다. 아래 기준에 따라 10점 만점으로 평가해주세요.

평가 기준:
1. 키워드 반영 여부
2. 동화 맥락의 일관성
3. 교훈의 명확성
4. 문법적 완성도
5. 비속어나 부적절한 표현의 유무
6. 서사 전개의 논리성
7. 반복되는 문장의 빈도

형식:
1. 키워드 반영 여부: X점 – [간단한 이유]
2. 동화 맥락의 일관성: X점 – ...
3. 교훈의 명확성: X점 – ...
4. 문법적 완성도: X점 – ...
5. 비속어나 부적절한 표현의 유무: X점 – ...
6. 서사 전개의 논리성: X점 – ...
7. 반복되는 문장의 빈도: X점 – ...

[동화 내용]
{story}
"""
    t0 = time.time()
    resp = model.generate_content(prompt)
    eval_time = time.time() - t0

    text = resp.text or ""
    scores = [float(s) for s in re.findall(r"(\d+(?:\.\d+)?)\s*점", text)]
    scores = scores[:7] if len(scores) >= 7 else scores + [0.0] * (7 - len(scores))

    avg_10 = sum(scores) / 7.0
    return {"raw_text": text, "scores": scores, "avg_10": avg_10, "eval_time_s": eval_time}

# 실행 예시
if __name__ == "__main__":
    name, age = "지훈", 6

    gen = AX4QLoRA(BASE_MODEL, LORA_PATH)
    result = gen.generate(name=name, age=age, max_new_tokens=320)
    story = result["story"]

    print("\n==== 동화 내용 ====\n")
    print(story[:1500] + ("\n...\n" if len(story) > 1500 else ""))

    # Gemini 평가
    try:
        eval_res = evaluate_with_gemini(story)
        print("\n==== Gemini 평가 요약 ====")
        for i, sc in enumerate(eval_res["scores"], 1):
            print(f"{i}. 항목 점수: {sc} / 10")
        print(f"- 평균 총점(10점 만점): {eval_res['avg_10']:.2f}")
        print(f("- 평가 시간: %.2f초" % eval_res['eval_time_s']))
    except Exception as e:
        print("[평가 오류]", e)

    # 성능 리포트
    print("\n==== 성능 리포트 ====")
    print(f"- 생성 시간: {result['gen_time_s']:.2f}초")
    print(f"- 생성 토큰 수: {result['new_tokens']}")
    print(f"- 생성 속도: {result['tps']:.2f} tok/s")

사용 중인 GPU: NVIDIA A100-SXM4-40GB


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[속도] 모델 로드 시간: 17.36초
- [GPU] 모델 로드 후 사용량: 19643.61 MB
- [GPU] 모델 로드 후 예약량: 34178.00 MB
- [GPU] 최대 사용량: 19887.11 MB

동화 생성 시간: 17.10초 | 생성 토큰: 194 | 속도: 11.35 tok/s
- [GPU] 생성 후 사용량: 19643.62 MB
- [GPU] 생성 후 예약량: 34190.00 MB
- [GPU] 최대 사용량: 19912.10 MB

==== 동화 내용 ====

옛날 옛적에 작은 마을에 지훈이라는 아이가 살고 있었어요. 지훈은 매일 아침 햇살이 뜨면 일어나 놀이터로 갔어요. 그곳에서 친구들과 함께 모래성을 쌓고, 그네를 타고, 미끄럼틀도 타곤 했어요.

어느 날, 지훈은 새로운 친구, 미나를 만났어요. 미나는 처음엔 조금 부끄러웠지만, 지훈이 따뜻하게 대해 주자 금방 친구가 되었어요. 두 친구는 함께 그림도 그리고, 간식을 나눠 먹으며 즐거운 시간을 보냈어요.

그러나 어느 날, 미나가 실수로 지훈의 그림을 망가뜨렸어요. 지훈은 화가 났지만, 미나에게 "괜찮아, 다음에는 더 잘 그릴 수 있어!"라고 말했어요. 미나도 사과하고, 둘은 다시 웃으며 그림을 새로 그렸어요.

이 이야기를 통해 지훈은 친구의 소중함을 배웠고, 실수를 해도 서로 이해하고 배려하는 것이 얼마나 중요한지 알게 되었어요. 그날 이후로 지훈과 미나는 더욱 좋은 친구가 되었답니다.

그리고 이 이야기는 우리에게 이렇게 알려줘요: "친구를 배려하고, 실수를 이해하면 더 행복한 친구가 될 수 있어요."

==== Gemini 평가 요약 ====
1. 항목 점수: 10.0 / 10
2. 항목 점수: 10.0 / 10
3. 항목 점수: 10.0 / 10
4. 항목 점수: 10.0 / 10
5. 항목 점수: 10.0 / 10
6. 항목 점수: 10.0 / 10
7. 항목 점수: 9.0 / 10
- 평균 총점(10점 만점): 9.86
[평가 오류] name 'f' is not define